<a href="https://colab.research.google.com/github/hariknair77/C-Programming/blob/master/fruitBinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! wget https://reboot20.s3.ap-south-1.amazonaws.com/apples.zip

--2020-02-29 04:55:29--  https://reboot20.s3.ap-south-1.amazonaws.com/apples.zip
Resolving reboot20.s3.ap-south-1.amazonaws.com (reboot20.s3.ap-south-1.amazonaws.com)... 52.219.64.103
Connecting to reboot20.s3.ap-south-1.amazonaws.com (reboot20.s3.ap-south-1.amazonaws.com)|52.219.64.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 669112720 (638M) [application/zip]
Saving to: ‘apples.zip’

apples.zip          100%[===================>] 638.12M  36.6MB/s    in 15s     

2020-02-29 04:55:44 (41.7 MB/s) - ‘apples.zip’ saved [669112720/669112720]



In [0]:
! unzip '/content/apples.zip'

Streaming output truncated to the last 5000 lines.
  inflating: apples/test_set/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 5.27.54 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 5.28.10 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 5.28.59 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.00.35 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.01.08 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.02.31 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.02.54 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.03.34 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 2018-06-08 at 5.03.40 PM.png  
  inflating: apples/test_set/freshapples/rotated_by_30_Screen Shot 

In [0]:
#check if runtype is in GPU mode for faster processing
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if device_name!='/device:GPU:0':
  raise SystemError('GPU device not found')
print('Fount GPU at: {}'.format(device_name))

Fount GPU at: /device:GPU:0


In [0]:
import keras#Keras is built over a basic layer of tensorflow which is much more complicated by itself
from keras.models import Sequential 
from keras.layers import Conv2D#from keras.layers import Convolution2D#Older name
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
#initialise CNN
classifier=Sequential()

In [0]:
#Step 1: Convolution Layer
classifier.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))#total 32 kernels in covolutional layer each of size 3x3, 3 indicates RGB image while 1 indicates Black and white image on gray scale

In [0]:
#Step 2: pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [0]:
classifier.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))#total 32 kernels in covolutional layer each of size 3x3, 3 indicates RGB image while 1 indicates Black and white image on gray scale
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [0]:
#Step 3: Flatten layer
classifier.add(Flatten())

In [0]:
#Step 4: ANN
classifier.add(Dense(activation='relu',units=128))
classifier.add(Dense(activation='sigmoid',units=1))

In [0]:
#compile CNN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [0]:
#Fitting the training data to the model
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)#Each of these transformations is applied to consecutive batches

In [0]:
test_datagen=ImageDataGenerator(rescale=1/255)

In [0]:
batchsize=64#Determines total processing time

In [0]:
training_set=train_datagen.flow_from_directory('/content/apples/train_set',target_size=(128,128),batch_size=batchsize,class_mode='binary')

Found 4035 images belonging to 2 classes.


In [0]:
test_set=test_datagen.flow_from_directory('/content/apples/test_set',target_size=(128,128),batch_size=batchsize,class_mode='binary')

Found 996 images belonging to 2 classes.


In [0]:
no_of_steps_per_epoch_training=int(training_set.n/batchsize)
no_of_steps_per_epoch_testing=int(test_set.n/batchsize)
print(no_of_steps_per_epoch_training)
print(no_of_steps_per_epoch_testing)

63
15


In [0]:
checkpoints=[]
checkpoints.append(keras.callbacks.ModelCheckpoint('test.h5',monitor='val_acccuracy',verbose=0,save_best_only=False,save_weights_only=False,mode='auto',period=1))

In [0]:
classifier.fit_generator(training_set,steps_per_epoch=no_of_steps_per_epoch_training,epochs=10,validation_data=test_set,validation_steps=no_of_steps_per_epoch_testing,callbacks=checkpoints)




Epoch 1/10





63/63 [==============================] - 36s 572ms/step - loss: 0.7466 - acc: 0.6693 - val_loss: 0.4993 - val_acc: 0.7271
Epoch 2/10
63/63 [==============================] - 28s 442ms/step - loss: 0.4016 - acc: 0.7966 - val_loss: 0.3070 - val_acc: 0.8852
Epoch 3/10
63/63 [==============================] - 28s 442ms/step - loss: 0.3052 - acc: 0.8924 - val_loss: 0.2771 - val_acc: 0.8723
Epoch 4/10
63/63 [==============================] - 27s 433ms/step - loss: 0.2267 - acc: 0.9032 - val_loss: 0.1902 - val_acc: 0.9227
Epoch 5/10
63/63 [==============================] - 27s 425ms/step - loss: 0.1824 - acc: 0.9298 - val_loss: 0.1566 - val_acc: 0.9335
Epoch 6/10
63/63 [==============================] - 27s 426ms/step - loss: 0.1645 - acc: 0.9323 - val_loss: 0.1767 - val_acc: 0.9152
Epoch 7/10
63/63 [==============================] - 26s 420ms/step - loss: 0.1761 - acc: 0.9296 - val_loss: 0.1571 - val_acc: 0.9431
Epoch 8/10
63/63 [==============================] - 26s 416ms

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
classifier.save('fruitBin.h5.2')

In [0]:
! cp -r fruitBin.h5.2 /content/drive/My\ Drive/Training\ Models

In [0]:
import numpy as np
from keras.preprocessing import image

In [0]:
test_image=image.load_img('/content/apppleFr.jfif',target_size=(128,128))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=classifier.predict(test_image)
if result==[[1.]]:
  print('rotton')
elif result==[[0.]]:
  print('fresh')

fresh


In [0]:
try:
  classifier.load_weights('/content/fruitBin.h5.2')
except:
  print("failed")

In [0]:
classifier.load_weights('/content/fruitBin.h5.2')